In [362]:
import pandas as pd
import json
"This is a string that makes this line exactly seventy-nine characters long."

def load_data(file_path, sheet_name):
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    return data

def extract_kyc_info(data):
    kyc_info = {}
    for i in range(0, 50, 5):
        customer_info = data.iloc[i:i+5]['KYC Information for 10 Customers:']
        customer_id = customer_info.iloc[0].split()[1].strip(':')
        name = customer_info.iloc[1].split(': ')[1]
        country = customer_info.iloc[2].split(': ')[1]
        purpose = customer_info.iloc[3].split(': ')[1]
        nature = customer_info.iloc[4].split(': ')[1]

        kyc_info[customer_id] = {
            'Customer': customer_id,
            'Name': name,
            'Country': country,
            'Purpose': purpose,
            'Nature': nature,
            'Transactions': {},
            'Number of Transactions': 0,
            'Risk Class': 'Not Classified'
        }
    return kyc_info

def append_transactions(data, kyc_info):
    for _, row in data.iterrows():
        customer_id = str(row['Customer_ID'])
        if customer_id in kyc_info:
            transaction_id = str(row['Transaction_ID'])
            transaction = {
                'Transaction_ID': row['Transaction_ID'],
                'Amount': row['Amount'],
                'Date': row['Date'].strftime('%Y-%m-%d'),
                'From/To Country': row['From/To Country'],
                'Transaction Type': row['Transaction_Type']
            }
            kyc_info[customer_id]['Transactions'][transaction_id] = transaction
            
    for customer_id in kyc_info:
        number_of_transactions = len(kyc_info[customer_id]['Transactions'])
    kyc_info[customer_id]['Number of Transactions'] = number_of_transactions

    return kyc_info


In [363]:
def classify_risk(customers):
    high_risk_countries = ['Russia', 'UAE', 'Oman']
    low_risk_countries = ['Sweden', 'Italy', 'Germany',
                          'Japan', 'France', 'Spain', 'USA']
    high_risk_purpose = ['trader', 'real estate', 'dealer', 'luxury']
    high_risk_transaction_types = ['Transfer']
    high_risk_nature = ['sporadic', 'large', 'irregular']
    
    risk_points = {}
    
    for customer_id, info in customers.items():
        risk_score = 0

        if info['Country'] in high_risk_countries:
            risk_score += 5
            
        if info['Country'] in low_risk_countries:
            risk_score -= 0.5

        if any(word in info['Purpose'].lower() for word in high_risk_purpose):
            risk_score += 2

        if any(word in info['Nature'].lower() for word in high_risk_nature):
            risk_score += 3.5

        for transaction in info['Transactions'].values():
            if transaction['Transaction Type'] in high_risk_transaction_types:
                risk_score += 0.5

        if risk_score >= 5:
            risk_class = 'High Risk'
        elif risk_score >= 4.5:
            risk_class = 'Normal Risk'
        else:
            risk_class = 'Low Risk'

        customers[customer_id]['Risk Class'] = risk_class
        risk_points[customer_id] = (risk_score, risk_class)
    
    return customers, risk_points


In [364]:
def main():
    file_path = '/Users/davidabaas/Downloads/Case data risk analyst AML 240607.xlsx'
    sheet_name = 'Sheet1'
    data = load_data(file_path, sheet_name)
    
    kyc_info = extract_kyc_info(data)
    transaction_data = data.reset_index(drop=True)
    kyc_info = append_transactions(transaction_data, kyc_info)
    
    kyc_info, risk_points = classify_risk(kyc_info)
    
    kyc_info_json = json.dumps(kyc_info, indent=4)
    print(kyc_info_json)

    
    
    return kyc_info

if __name__ == "__main__":
    kyc_info = main()

{
    "1001": {
        "Customer": "1001",
        "Name": "John Doe",
        "Country": "Sweden",
        "Purpose": "Personal savings and investments",
        "Nature": "Monthly deposits around $5,000, occasional withdrawals",
        "Transactions": {
            "1": {
                "Transaction_ID": 1,
                "Amount": 5000,
                "Date": "2023-03-01",
                "From/To Country": "Sweden",
                "Transaction Type": "Deposit"
            },
            "11": {
                "Transaction_ID": 11,
                "Amount": 4000,
                "Date": "2023-03-11",
                "From/To Country": "Sweden",
                "Transaction Type": "Deposit"
            },
            "21": {
                "Transaction_ID": 21,
                "Amount": 5000,
                "Date": "2023-03-21",
                "From/To Country": "Sweden",
                "Transaction Type": "Deposit"
            },
            "31": {
                "Trans

In [365]:
#print(json.dumps(kyc_info['1007']['Transactions'], indent = 4))
Customer_ID = '1007'
print(json.dumps(kyc_info[Customer_ID], indent = 4))

{
    "Customer": "1007",
    "Name": "Pierre Dubois",
    "Country": "France",
    "Purpose": "Art dealer",
    "Nature": "Large irregular deposits, $20,000 - $50,000",
    "Transactions": {
        "7": {
            "Transaction_ID": 7,
            "Amount": 40000,
            "Date": "2023-03-07",
            "From/To Country": "France",
            "Transaction Type": "Deposit"
        },
        "17": {
            "Transaction_ID": 17,
            "Amount": 30000,
            "Date": "2023-03-17",
            "From/To Country": "France",
            "Transaction Type": "Deposit"
        },
        "27": {
            "Transaction_ID": 27,
            "Amount": 50000,
            "Date": "2023-03-27",
            "From/To Country": "France",
            "Transaction Type": "Withdrawal"
        },
        "37": {
            "Transaction_ID": 37,
            "Amount": 32000,
            "Date": "2023-04-07",
            "From/To Country": "France",
            "Transaction Type": 

In [366]:
print(json.dumps(kyc_info['1007']['Transactions']['97']))

{"Transaction_ID": 97, "Amount": 29000, "Date": "2023-06-06", "From/To Country": "France", "Transaction Type": "Deposit"}


In [367]:
Transaction_ID = '97'
kyc_info['1007']['Transactions'][Transaction_ID]

{'Transaction_ID': 97,
 'Amount': 29000,
 'Date': '2023-06-06',
 'From/To Country': 'France',
 'Transaction Type': 'Deposit'}